In [1]:
#Import libraries for web-scraping and saving to CSV file.
import requests
import bs4
import re
import csv
import os
import pandas as pd
from Scraper import get_urls, events, fights, fightstats, fighters, upcoming_events, normalise_tables
from datetime import datetime
import time
import inspect
import logging
from functools import wraps

### get fight urls from event urls

In [2]:
main_url = requests.get('http://ufcstats.com/statistics/events/completed?page=all')
main_event_soup = bs4.BeautifulSoup(main_url.text, 'lxml')


#Adds href to list if href contains a link with keyword 'event-details'
all_event_urls = [item.get('href') for item in  main_event_soup.find_all('a') 
                    if type(item.get('href')) == str 
                    and 'event-details' in item.get('href')]

all_event_urls = all_event_urls[:2]


In [3]:
#Iterates through each event URL
all_fight_urls = []
for url in all_event_urls:
    event_url = requests.get(url)
    event_soup = bs4.BeautifulSoup(event_url.text,'lxml')

    #Scrapes fight URLs from event pages and adds to list
    for item in event_soup.find_all('a', class_='b-flag b-flag_style_green'):
        all_fight_urls.append(item.get('href'))

### get fight info from fight urls

In [4]:
def safe_fight_info_get(default_return=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except (IndexError, AttributeError, TypeError) as e:
                logging.warning(f"Known error for {func.__name__}: {type(e).__name__}: {e}")
                return default_return
            except Exception as e:
                logging.error(f"Unexpected error for {func.__name__}: {type(e).__name__}: {e}")
                return default_return
        return wrapper
    return decorator

@safe_fight_info_get(default_return='NULL')
def get_referee(overview):
    return overview[3].text.split(':')[1]

@safe_fight_info_get(default_return=('NULL','NULL'))
def get_fighters(fight_details,fight_soup):
    try:
        return fight_details[0].text, fight_details[1].text
    except:
        links = fight_soup.select('a.b-fight-details__person-link')
        return links[0].text, links[1].text

@safe_fight_info_get(default_return=('NULL','NULL'))
def get_aliases(nicknames):
    n1 = nicknames[0].text if len(nicknames[0].text.strip())>0 else 'NULL'
    n2 = nicknames[1].text if len(nicknames[1].text.strip())>0 else 'NULL'
    return n1,n2
    
        
@safe_fight_info_get(default_return='NULL')
def get_winner(win_lose,f_1,f_2):
    if (win_lose[0].text.strip()=='W') | (win_lose[1].text.strip()=='W'):
        if (win_lose[0].text.strip()=='W'):
            return f_1
        else:
            return f_2
    else:
        return 'NULL'

@safe_fight_info_get(default_return='NULL')
def get_title_fight(fight_type):
    if 'Title' in fight_type[0].text:
        return 'T'
    else:
        return 'F'

@safe_fight_info_get(default_return='NULL')
def get_weight_class(fight_type):
    if 'Light Heavyweight' in fight_type[0].text.strip():
        return 'Light Heavyweight'
        
    elif 'Women' in fight_type[0].text.strip():
        return "Women's " + re.findall('\w*weight',fight_type[0].text.strip())[0]
        
    elif 'Catch Weight' in fight_type[0].text.strip():
        return 'Catch Weight'
            
    elif 'Open Weight' in fight_type[0].text.strip():
        return 'Open Weight' 
    else:   
        try:
            return re.findall('\w*weight',fight_type[0].text.strip())[0]
        except: 
            return 'NULL'

@safe_fight_info_get(default_return='NULL')
def get_gender(fight_type):
    if 'women' in str.lower(fight_type[0].text):
        return 'F'
    else:
        return 'M'

@safe_fight_info_get(default_return='NULL')
def get_result(select_result,select_result_details):
    if 'Decision' in select_result[0].text.split(':')[1]:
        return select_result[0].text.split(':')[1].split()[0], select_result[0].text.split(':')[1].split()[-1]
    else:
        return select_result[0].text.split(':')[1], select_result_details[1].text.split(':')[-1]

<>:58: SyntaxWarning: invalid escape sequence '\w'
<>:67: SyntaxWarning: invalid escape sequence '\w'
<>:58: SyntaxWarning: invalid escape sequence '\w'
<>:67: SyntaxWarning: invalid escape sequence '\w'
C:\Users\RautaN01\AppData\Local\Temp\ipykernel_16520\1259399813.py:58: SyntaxWarning: invalid escape sequence '\w'
  return "Women's " + re.findall('\w*weight',fight_type[0].text.strip())[0]
C:\Users\RautaN01\AppData\Local\Temp\ipykernel_16520\1259399813.py:67: SyntaxWarning: invalid escape sequence '\w'
  return re.findall('\w*weight',fight_type[0].text.strip())[0]


In [5]:
all_fight_urls

['http://ufcstats.com/fight-details/633f4b5ec767769d',
 'http://ufcstats.com/fight-details/a526a72ade275f46',
 'http://ufcstats.com/fight-details/4072abf0b0c179f2',
 'http://ufcstats.com/fight-details/9bbd45774fb4a6eb',
 'http://ufcstats.com/fight-details/fbf83609dc96f32e',
 'http://ufcstats.com/fight-details/62fbdd4f0bcd3c2a',
 'http://ufcstats.com/fight-details/d8d22dcc1f6912e7',
 'http://ufcstats.com/fight-details/97770b38df67b276',
 'http://ufcstats.com/fight-details/6f3cb7ce84185dd3',
 'http://ufcstats.com/fight-details/7d51649373d42571',
 'http://ufcstats.com/fight-details/c2c20a04a78d9143',
 'http://ufcstats.com/fight-details/e7b3ca036835b7c2',
 'http://ufcstats.com/fight-details/23201d09a78d446d',
 'http://ufcstats.com/fight-details/7fd6770fe2d4f258']

In [6]:
#Scrapes details of each UFC fight and appends to file 'ufc_fight_data.csv'

url = all_fight_urls[0]

fight_url = requests.get(url)
fight_soup = bs4.BeautifulSoup(fight_url.text,'lxml')

#Define key select statements
overview = fight_soup.select('i.b-fight-details__text-item')
select_result = fight_soup.select('i.b-fight-details__text-item_first')
select_result_details = fight_soup.select('p.b-fight-details__text')
fight_details = fight_soup.select('p.b-fight-details__table-text')
fight_type = fight_soup.select('i.b-fight-details__fight-title')
win_lose = fight_soup.select('i.b-fight-details__person-status')
# nicknames = fight_soup.select('p.b-fight-details__person-title')


In [7]:
#Scrape fight details
event_name = fight_soup.h2.text
referee = get_referee(overview)
f_1,f_2 = get_fighters(fight_details,fight_soup)
# f_1_alias,f_2_alias= get_aliases(nicknames)
num_rounds = overview[2].text.split(':')[1].strip()[0]
title_fight = get_title_fight(fight_type)
weight_class = get_weight_class(fight_type)
gender = get_gender(fight_type)  
result,result_details = get_result(select_result,select_result_details)
finish_round = overview[0].text.split(':')[1]
finish_time = re.findall('\d:\d\d',overview[1].text)[0]
winner = get_winner(win_lose,f_1,f_2)

<>:12: SyntaxWarning: invalid escape sequence '\d'
<>:12: SyntaxWarning: invalid escape sequence '\d'
C:\Users\RautaN01\AppData\Local\Temp\ipykernel_16520\3913756993.py:12: SyntaxWarning: invalid escape sequence '\d'
  finish_time = re.findall('\d:\d\d',overview[1].text)[0]


In [8]:
field_list = [
    event_name,
    referee,
    f_1,
    f_2,
    # f_1_alias, - this is stored at the fighter level as there is only 1 relevant nickname per fighter
    # f_2_alias,
    num_rounds,
    title_fight,
    weight_class,
    gender,
    result,
    result_details,
    finish_round,
    finish_time,
    winner
]

In [9]:
[i.strip() for i in field_list]

['UFC Fight Night: Machado Garry vs. Prates',
 'Dan Miragliotta',
 'Ian Machado Garry',
 'Carlos Prates',
 '5',
 'F',
 'Welterweight',
 'M',
 'Decision',
 'Unanimous',
 '5',
 '5:00',
 'Ian Machado Garry']

### get fight stat info from fight urls

In [31]:
def safe_text(stat_list, index, default = 'NULL', split = False, part = None, logger = None):
    try:
        text = stat_list[index].text
        if split:
            parts = text.split(' of ')
            return parts[part] if len(parts) == 2 else default
        return text
    except Exception as e:
        if logger:
            logger.warning(f"Failed to get a stat at index {index}:{e}")
        return default


def get_fighter_id(fight_soup,fight_stats,fighter):
    try:
        return fight_stats[fighter - 1].text
    except:
        try:
            return fight_soup.select('a.b-fight-details __person-link')[fighter - 1].text
        except:
            return 'NULL'
        
        
def get_total_stats(fight_stats,fighter,round = 0):

    idx = fighter + (round * 20)

    kd = 1 + idx
    total = 7 + idx
    significant = 3 + idx
    td_index = 9 + idx
    submission_index = 13 + idx
    reversal_index = 15 + idx
    control_index = 17 + idx
    return(
        safe_text(fight_stats, kd),
        safe_text(fight_stats, total, split = True, part = 1),
        safe_text(fight_stats, total, split = True, part = 0),
        safe_text(fight_stats, significant, split = True, part = 1),
        safe_text(fight_stats, significant, split = True, part = 0),
        safe_text(fight_stats, td_index, split = True, part = 1),
        safe_text(fight_stats, td_index, split = True, part = 0),
        safe_text(fight_stats, submission_index),
        safe_text(fight_stats, reversal_index),
        safe_text(fight_stats, control_index)
    )


def get_strike_stats_distributed(fight_stats,fighter,round=0):
    
    idx = fighter + (round * 18)

    indexes = {
    'hs_index' : 5 + idx,
    'bs_index' : 7 + idx,
    'ls_index' : 9 + idx,
    'ds_index' : 11 + idx,
    'cs_index' : 13 + idx,
    'gs_index' : 15 + idx
    }

    results = []
    for key, val in indexes.items():
        for i in range(2):
            results.append(safe_text(fight_stats,val,split=True,part=i))
 
    return tuple(results)

In [32]:

fight_stats_table_totals = fight_soup.select('p.b-fight-details__table-text')
if not fight_stats_table_totals:
    logging.error(f'Fight stats totals table not found. url {url}')

fight_stats_table_significant_strikes = fight_soup.find(
            "p",
            class_="b-fight-details__collapse-link_tot",
            string=lambda x: x and "Significant Strikes" in x
        ).find_next('tbody', class_='b-fight-details__table-body').select('p.b-fight-details__table-text')
if not fight_stats_table_significant_strikes:
    logging.error(f'Fight Details table not found. url {url}')

fight_stats_table_totals_rounds = fight_soup.find(
            "i",
            class_="b-fight-details__collapse-left"
        ).find_next("tbody","b-fight-details__table-body").select("p.b-fight-details__table-text")
if not fight_stats_table_totals_rounds:
    logging.error(f'Fight Round totals table not found. url {url}')

fight_stats_table_significant_strikes_rounds = fight_soup.find(
            "i",
            class_="b-fight-details__collapse-left"
        ).find_next(
            "i",
            class_="b-fight-details__collapse-left"
        ).find_next("tbody","b-fight-details__table-body").select("p.b-fight-details__table-text")
if not fight_stats_table_significant_strikes_rounds:
    logging.error(f'Fight Round Details table not found. url {url}')

fight_stats = {
    "url" : url,
    "fighter_1":get_fighter_id(fight_soup, fight_stats_table_totals,1),
    "fighter_2":get_fighter_id(fight_soup, fight_stats_table_totals,2)
}
for fighter in [1,2]:
    (fight_stats[f"fighter_{fighter}_knockdowns"],
    fight_stats[f"fighter_{fighter}_total_strikes_att"],
    fight_stats[f"fighter_{fighter}_total_strikes_succ"],
    fight_stats[f"fighter_{fighter}_sig_strikes_att"],
    fight_stats[f"fighter_{fighter}_sig_strikes_succ"],
    fight_stats[f"fighter_{fighter}_takedown_att"],
    fight_stats[f"fighter_{fighter}_takedown_succ"],
    fight_stats[f"fighter_{fighter}_submission_att"],
    fight_stats[f"fighter_{fighter}_reversals"],
    fight_stats[f"fighter_{fighter}_ctrl_time"]) = get_total_stats(fight_stats_table_totals, fighter)
    
    (fight_stats[f"fighter_{fighter}_head_strikes_att"],
    fight_stats[f"fighter_{fighter}_head_strikes_succ"],
    fight_stats[f"fighter_{fighter}_body_strikes_att"],
    fight_stats[f"fighter_{fighter}_body_strikes_succ"],
    fight_stats[f"fighter_{fighter}_leg_strikes_att"],
    fight_stats[f"fighter_{fighter}_leg_strikes_succ"],
    fight_stats[f"fighter_{fighter}_distance_strikes_att"],
    fight_stats[f"fighter_{fighter}_distance_strikes_succ"],
    fight_stats[f"fighter_{fighter}_clinch_strikes_att"],
    fight_stats[f"fighter_{fighter}_clinch_strikes_succ"],
    fight_stats[f"fighter_{fighter}_ground_strikes_att"],
    fight_stats[f"fighter_{fighter}_ground_strikes_succ"]) = get_strike_stats_distributed(fight_stats_table_significant_strikes, fighter)
    
    # per round stats
    for round in range(5):
        round_label = round + 1
        (fight_stats[f"fighter_{fighter}_round_{round_label}_knockdowns"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_total_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_total_strikes_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_sig_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_sig_strikes_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_takedown_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_takedown_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_submission_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_reversals"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_ctrl_time"]) = get_total_stats(fight_stats_table_totals_rounds, fighter, round = round)
    
        (fight_stats[f"fighter_{fighter}_round_{round_label}_head_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_head_strikes_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_body_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_body_strikes_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_leg_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_leg_strikes_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_distance_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_distance_strikes_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_clinch_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_clinch_strikes_succ"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_ground_strikes_att"],
        fight_stats[f"fighter_{fighter}_round_{round_label}_ground_strikes_succ"]) = get_strike_stats_distributed(fight_stats_table_significant_strikes_rounds, fighter, round = round)




In [33]:
for key,value in fight_stats.items():
    fight_stats[key] = value.strip()

In [37]:
pd.DataFrame([fight_stats]).info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 267 columns):
 #    Column                                   Non-Null Count  Dtype 
---   ------                                   --------------  ----- 
 0    url                                      1 non-null      object
 1    fighter_1                                1 non-null      object
 2    fighter_2                                1 non-null      object
 3    fighter_1_knockdowns                     1 non-null      object
 4    fighter_1_total_strikes_att              1 non-null      object
 5    fighter_1_total_strikes_succ             1 non-null      object
 6    fighter_1_sig_strikes_att                1 non-null      object
 7    fighter_1_sig_strikes_succ               1 non-null      object
 8    fighter_1_takedown_att                   1 non-null      object
 9    fighter_1_takedown_succ                  1 non-null      object
 10   fighter_1_submission_att                 1 non-null 

In [38]:
pd.DataFrame([fight_stats]).to_csv('test.csv')